<a href="https://colab.research.google.com/github/piaoyanxiu/vocab-collocation/blob/main/%EC%9B%B9%ED%85%8D%EB%A7%88_%ED%94%8C%EC%A0%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=966de50cbfc376053ecc753c6b2d9e4056981fd3c36de55663f4161a6a8a495e
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
import numpy as np
import pandas as pd

import requests
import json

from collections import Counter

from wordcloud import WordCloud

from bs4 import BeautifulSoup as bs

import wget

import warnings
warnings.filterwarnings('ignore')

In [10]:
res = requests.get('https://www.suneung.re.kr/boardCnts/list.do?boardID=1500236&m=0403&s=suneung&searchStr=')

print("◎ 응답 코드: ", res.status_code)

◎ 응답 코드:  200


In [12]:
soup = bs(res.text)

In [15]:
for contents in soup.find_all('a'):
  if 'title' in contents.attrs:
    if "영어" in contents.attrs['title']:
      if 'onclick' in contents.attrs :
        print(contents)

<a href="#;" onclick="fn_fileDown('6ae9013673bcc7226d19d536d337733c');" title="영어영역_문제지.pdf"><img alt="영어영역_문제지.pdf" height="16" src="/images/board/board_pdf.png" width="16"/> </a>
<a href="#;" onclick="fn_fileDown('a865be83814ad0de8806d3c5d6aef76c');" title="영어영역듣기평가음원.zip"><img alt="영어영역듣기평가음원.zip" height="16" src="/images/board/board_zip.png" width="16"/> </a>
<a href="#;" onclick="fn_fileDown('a8e37ec95dcf0c8a1640f198a1f4d6ff');" title="영어영역듣기평가대본.pdf"><img alt="영어영역듣기평가대본.pdf" height="16" src="/images/board/board_pdf.png" width="16"/> </a>
<a href="#;" onclick="fn_fileDown('b589da9509036b93560087955c79d5b0');" title="영어영역_정답표.pdf"><img alt="영어영역_정답표.pdf" height="16" src="/images/board/board_pdf.png" width="16"/> </a>


In [21]:
links=[]

def web_scraping(maxpage, start = 1):

    maxpage=int(maxpage)
    print ('maxpage = ', maxpage)
    print()

    while 1:

        if start >= maxpage+1 :
          break

        org_url='https://www.kice.re.kr/boardCnts/'
        url='https://www.suneung.re.kr/boardCnts/list.do?type=default&page={}&searchStr=&m=0403&C06=&boardID=1500236&C05=&C04=&C03=&C02=&searchType=S&C01=&s=suneung'.format(start)
        req = requests.get(url)

        page = bs(req.text, 'html.parser')

        for link in page.find_all('a'):
            title=link.get('title')
            if (title=="3교시_영어영역_문제지.pdf") or (title=="3교시_영어_문제지.pdf") or (title=="영어영역_문제지.pdf"):
               current_link=link.get('onclick')

               if current_link is None:
                 continue

               if current_link.startswith('fn'):
                  current_link=current_link.strip("');")
                  current_link=current_link.replace("fn_fileDown('", "fileDown.do?fileSeq=")

                  links.append(org_url+current_link)
                  print("currentLink", current_link)
               else: False

        start += 1

    for link in links:
          wget.download(link)


In [22]:
maxpage = input("◎ 검색할 최대 페이지수: ")
print()

web_scraping(maxpage)

◎ 검색할 최대 페이지수: 10

maxpage =  10

currentLink fileDown.do?fileSeq=6ae9013673bcc7226d19d536d337733c
currentLink fileDown.do?fileSeq=59c8b2f202f30c56fdd1cc1931188bad
currentLink fileDown.do?fileSeq=e04e90133b6f6553fecb1d8b78f91ad1
currentLink fileDown.do?fileSeq=4f7b44eb981a8794fc79c490f7b4c01e
currentLink fileDown.do?fileSeq=de4823f124caf56f82e18bba901f4159
currentLink fileDown.do?fileSeq=daa793a99c83fa0ed1e4d11fb5203ad4
currentLink fileDown.do?fileSeq=bffd4ecdecf411375d124d5cf8a934ca
currentLink fileDown.do?fileSeq=1fa497168518a43c55753f7860d62a65
currentLink fileDown.do?fileSeq=54419bf4091095ae780c57e8de51953a
currentLink fileDown.do?fileSeq=91c10e0c7df16f84a32cd96beb2120fe
currentLink fileDown.do?fileSeq=a98c125c9412db4fd89b322f93c7f908
currentLink fileDown.do?fileSeq=55c0e33363dfbdca3e8616460dcab736


In [18]:
from google.colab import drive
drive.mount('/content/gdrive/')

file_path = "/content/"

Mounted at /content/gdrive/


In [19]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 30.6 MB/s eta 0:00:00


In [25]:
#/content/3%EA%B5%90%EC%8B%9C_%EC%98%81%EC%96%B4%EC%98%81%EC%97%AD_%EB%AC%B8%EC%A0%9C%EC%A7%80 (1).pdf
#/content/3%EA%B5%90%EC%8B%9C_%EC%98%81%EC%96%B4%EC%98%81%EC%97%AD_%EB%AC%B8%EC%A0%9C%EC%A7%80 (2).pdf

import os
import fitz
import re

filename=[]
engl_text=[]
texts='a'

file_list=os.listdir(file_path)
#print(file_list)

for file in file_list:
  if file.endswith('.pdf'):
    filepath=os.path.join(file_path, file)
    data=fitz.open(filepath)
    for page in data:
      text=page.get_text()
      engl_text=re.sub('[^a-zA-Z ]',' ',text)
      texts=texts+engl_text


print(len(texts))

308173


In [24]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

c_vector = CountVectorizer()
engl_text=[engl_text]
c_vector.fit_transform(engl_text)

print('▶ 출현 단어: ', c_vector.get_feature_names_out())
print(len(c_vector.get_feature_names_out()))

▶ 출현 단어:  ['able' 'access' 'ace' 'additional' 'addressed' 'advised' 'after' 'all'
 'allocate' 'allocation' 'also' 'am' 'amusement' 'an' 'and' 'andes'
 'applied' 'are' 'arises' 'as' 'ashamed' 'asked' 'associated' 'at'
 'attentively' 'auction' 'away' 'be' 'become' 'been' 'bought' 'brag'
 'bragged' 'bribe' 'bribed' 'but' 'by' 'bypassed' 'can' 'capture'
 'classmate' 'collect' 'combined' 'communities' 'community'
 'congratulated' 'congratulations' 'constitutes' 'contest' 'crop' 'crops'
 'cultivation' 'dad' 'debates' 'decided' 'decrease' 'delivered' 'derived'
 'desert' 'development' 'disappointed' 'disclosed' 'do' 'don' 'done'
 'economic' 'end' 'ensures' 'ensuring' 'envelope' 'essay' 'etc' 'example'
 'excited' 'eyes' 'fair' 'favours' 'feel' 'felt' 'fill' 'first' 'flash'
 'floods' 'foolish' 'for' 'found' 'free' 'from' 'front' 'full' 'go' 'gone'
 'government' 'grants' 'grow' 'guaranteed' 'had' 'hadn' 'handed'
 'harvested' 'has' 'have' 'he' 'hear' 'heard' 'her' 'hero' 'hesitant'
 'him' 'his' 'h

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import nltk

nltk.download('book')

from nltk.tokenize import RegexpTokenizer

In [29]:
hfw_url='https://sightwords.com/sight-words/fry/'

hfw_req = requests.get(hfw_url)

hfw_page = bs(hfw_req.text, 'html.parser')


In [31]:

hfw = hfw_page.find_all(attrs='pdf')[0]

#print(hfw)

grade1=hfw.find_all('li')[0].get_text()
grade2=hfw.find_all('li')[1].get_text()
grade3=hfw.find_all('li')[2].get_text()

#print(grade)

grades=grade1+grade2+grade3

#hfws=grade.get_text()

hfws=str(grades)
hfws=re.sub('[^a-zA-Z ]','',hfws)
hfws=hfws.replace("Fry Sight Words","")


In [32]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

hfw_tokens=word_tokenize(hfws)

stop_words = set(stopwords.words('english'))
alphabet=['a','b','c', 'd', 'e']
number=['one','two','three']

texts_lower=texts.lower()

word_tokens = word_tokenize(texts_lower)

result = []
for word in word_tokens:
    if word not in stop_words:
      if word not in alphabet:
        if word not in number:
          if word not in hfw_tokens:
            result.append(word)

print(len(result))

19541


In [33]:
from nltk.tag import pos_tag

tagged_list = pos_tag(result)

nouns_list = [tag[0] for tag in tagged_list if tag[1] == "NNS"]
verbs_list = [tag[0] for tag in tagged_list if tag[1] == "VBD" or tag[1] == "VBG" or tag[1] == "VBN" or tag[1] == "VBP" or tag[1] == "VBZ"]
adj_list=[tag[0] for tag in tagged_list if tag[1] == "JJS"]
others_list = [tag[0] for tag in tagged_list if tag[1] != "NNS" and tag[1] != "VBD" and tag[1] != "VBG" and tag[1] != "VBN" and tag[1] != "VBP" and tag[1] != "VBZ" and tag[1] != "JJS"]


In [34]:
from nltk.stem import WordNetLemmatizer

wnlm = WordNetLemmatizer()

noun_l=[wnlm.lemmatize(wd, pos="n") for wd in nouns_list]
verb_l=[wnlm.lemmatize(wd, pos="v") for wd in verbs_list]
adj_l=[wnlm.lemmatize(wd, pos="a") for wd in adj_list]

lemmas=noun_l+verb_l+adj_l+others_list

len(lemmas)


19541

In [35]:
from collections import Counter

counts = Counter(lemmas)

words = counts.most_common(100)

print(words)

[('social', 82), ('information', 67), ('become', 66), ('human', 59), ('process', 59), ('individual', 52), ('experience', 51), ('problem', 47), ('however', 45), ('cultural', 44), ('make', 41), ('others', 40), ('culture', 40), ('activity', 40), ('age', 40), ('provide', 40), ('product', 39), ('sally', 38), ('music', 37), ('student', 34), ('way', 33), ('material', 33), ('value', 33), ('know', 33), ('give', 32), ('room', 31), ('feel', 31), ('cause', 30), ('result', 30), ('animal', 30), ('view', 30), ('art', 30), ('present', 30), ('p', 30), ('look', 30), ('better', 30), ('effect', 29), ('plant', 29), ('data', 29), ('service', 29), ('decision', 29), ('really', 29), ('benefit', 28), ('event', 28), ('system', 28), ('performance', 28), ('allow', 28), ('consider', 28), ('share', 28), ('rather', 28), ('science', 27), ('class', 27), ('understand', 27), ('require', 27), ('use', 27), ('knowledge', 27), ('self', 27), ('call', 26), ('base', 26), ('include', 26), ('ask', 26), ('best', 26), ('particular'

In [36]:
challenge=counts.most_common(1000)
challenge[-30:]

[('creation', 5),
 ('ethic', 5),
 ('food', 5),
 ('regulation', 5),
 ('superior', 5),
 ('assumes', 5),
 ('grain', 5),
 ('anthropologist', 5),
 ('domain', 5),
 ('root', 5),
 ('habit', 5),
 ('rate', 5),
 ('end', 5),
 ('chain', 5),
 ('mechanism', 5),
 ('attack', 5),
 ('economics', 5),
 ('treatment', 5),
 ('expectation', 5),
 ('outcome', 5),
 ('destination', 5),
 ('machine', 5),
 ('quality', 5),
 ('ecosystem', 5),
 ('mammal', 5),
 ('consideration', 5),
 ('character', 5),
 ('bag', 5),
 ('stone', 5),
 ('pool', 5)]